# Neurohackademy Hackathon: Extracting Emotion-ROI Based Tracks
## hcp-des-emotional-states-project
#### Author: Diana Hobbs, +Ariel Rokem, +ChatGPT/stackoverflow/etc
#### Date: August 2024

In [1]:
# import libraries
import os
import csv
import dipy
import abagen 
import pandas as pd
import nibabel as nib

from dipy.io.streamline import load_trk
# from utilities import ls, crawl
from pathlib import Path
from cloudpathlib import S3Path, S3Client

In [2]:
# # get to base 
# !cd
# !ls

In [3]:
# # create .aws/credentials with access key ID and secret access key
# !mkdir -p ~/.aws
# !cd .aws
# !touch credentials

# # from NH course 
# !echo '[hcp]' > ~/.aws/credentials
# !echo 'aws_access_key_id = <<INPUT YOUR ACCESS KEY ID HERE>>' >> ~/.aws/credentials
# !echo 'aws_secret_access_key = <<INPUT YOUR SECRET ACCESS KEY HERE>>' >> ~/.aws/credentials
# !echo ' ' = '   ' >> ~/.aws/credentials # **MUST add a blank line after or it gets wonky

In [4]:
# from Ariel & NH course
# Make sure that we have a cache path:
cache_path = Path('/tmp/cache')
# Just because we have made a cache path object doesn't mean that the directory
# we made exists; here we check if it exists and make the directory if not.
if not cache_path.exists():
    cache_path.mkdir()

client = S3Client(
    local_cache_dir=cache_path,
    no_sign_request=True)

hcp_base_path = S3Path(
    "s3://open-neurodata", # **NOTE: we are using open-neurodata, not hcp-openaccess
    client=client)
hcp_subdirs = list(hcp_base_path.iterdir())
# There are about 1200 of these, so we won't show them all, just the first 10:
hcp_subdirs[:10]

[S3Path('s3://open-neurodata/ABagemouse'),
 S3Path('s3://open-neurodata/ARA'),
 S3Path('s3://open-neurodata/ailey'),
 S3Path('s3://open-neurodata/allan_johnson'),
 S3Path('s3://open-neurodata/ara_2016'),
 S3Path('s3://open-neurodata/bhatla'),
 S3Path('s3://open-neurodata/bigbrain'),
 S3Path('s3://open-neurodata/bloss'),
 S3Path('s3://open-neurodata/bock11'),
 S3Path('s3://open-neurodata/brainlit')]

In [5]:
# navigate to connectomes & extract DKT parcellated tracts
# crawl(hcp_base_path / "hcp1200" / "sub-100206" / "ses-1" / "connectomes" )
# crawl(hcp_base_path / "hcp1200" / "sub-100206" / "ses-1" / "connectomes" / "DKT_space-MNI152NLin6_res-1x1x1")

## Loading DKT parcellation labels

In [6]:
# get dkt ROI labels
dkt = abagen.datasets.fetch_desikan_killiany()
dkt_image = dkt["image"]
dkt_info = dkt["info"]

# load labels
labels = pd.read_csv(dkt_info, sep=",")
print(labels.shape)
labels.index=range(1, len(labels)+1)
print(labels)

# concatenate label/hemi/structure for ease of use
labels['hemi_label'] = labels['hemisphere'] + "_" + labels['label'] + "_" + labels['structure']

# select id / hemi_label
hemi_labels = labels.iloc[:, [0, 4]]
print(hemi_labels)

(83, 4)
    id                    label hemisphere            structure
1    1                 bankssts          L               cortex
2    2  caudalanteriorcingulate          L               cortex
3    3      caudalmiddlefrontal          L               cortex
4    4                   cuneus          L               cortex
5    5               entorhinal          L               cortex
..  ..                      ...        ...                  ...
79  79                 pallidum          R  subcortex/brainstem
80  80            accumbensarea          R  subcortex/brainstem
81  81              hippocampus          R  subcortex/brainstem
82  82                 amygdala          R  subcortex/brainstem
83  83                brainstem          B  subcortex/brainstem

[83 rows x 4 columns]
    id                           hemi_label
1    1                    L_bankssts_cortex
2    2     L_caudalanteriorcingulate_cortex
3    3         L_caudalmiddlefrontal_cortex
4    4                   

In [7]:
# chatCPT extraordinaire 
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)   # None means no limit
pd.set_option('display.max_columns', None) # None means no limit
pd.set_option('display.expand_frame_repr', False) # Prevent frame expansion

In [8]:
print(hemi_labels["hemi_label"].all)

<bound method NDFrame._add_numeric_operations.<locals>.all of 1                        L_bankssts_cortex
2         L_caudalanteriorcingulate_cortex
3             L_caudalmiddlefrontal_cortex
4                          L_cuneus_cortex
5                      L_entorhinal_cortex
6                        L_fusiform_cortex
7                L_inferiorparietal_cortex
8                L_inferiortemporal_cortex
9                L_isthmuscingulate_cortex
10               L_lateraloccipital_cortex
11           L_lateralorbitofrontal_cortex
12                        L_lingual_cortex
13            L_medialorbitofrontal_cortex
14                 L_middletemporal_cortex
15                L_parahippocampal_cortex
16                    L_paracentral_cortex
17                L_parsopercularis_cortex
18                  L_parsorbitalis_cortex
19               L_parstriangularis_cortex
20                  L_pericalcarine_cortex
21                    L_postcentral_cortex
22             L_posteriorcingulate

In [9]:
# isolate roi labels

# caudalanteriorcingulate L: 2
# caudalanteriorcingulate R: 43
# rostralanteriorcingulate L: 25
# rostralanteriorcingulate R: 66
# lateralorbitofrontal L:11
# lateralorbitofrontal R:52
# medialorbitofrontal L:13
# medialorbitofrontal R:54
# amygdala L: 41
# amygdala R: 82
# thalamus L: 35
# thalamus R: 76
# hippocampus L: 40
# hippocampus R: 81

# selected_rois =  [2,43,25,66,11,52,13,54,41,82] ## initially amy<>orbitofronal amy<>ACC
# selected_rois =  [2,43,25,66,11,52,13,54,41,82,40,81,35,76] # added in hippocampus and thalamus
selected_rois =  [25,66,41,82,40,81,35,76] # removed orbitofrontal & caudal ACC

# 
selected_roi_labels = hemi_labels[hemi_labels['id'].isin(selected_rois)]
print(selected_roi_labels)

## initially amy<>orbitofronal amy<>ACC
# region_dict = {  2: "L_caudalanteriorcingulate_cortex",
#                 43: "R_caudalanteriorcingulate_cortex",
#                 25: "L_rostralanteriorcingulate_cortex",
#                 66: "R_rostralanteriorcingulate_cortex",
#                 11: "L_lateralorbitofrontal_cortex", 
#                 52: "R_lateralorbitofrontal_cortex", 
#                 13: "L_medialorbitofrontal_cortex",
#                 54: "R_medialorbitofrontal_cortex",
#                 41: "L_amygdala",
#                 82: "R_amygdala"  }

# added in hippocampus and thalamus
# region_dict = {  2: "L_caudalanteriorcingulate_cortex",
#                 43: "R_caudalanteriorcingulate_cortex",
#                 25: "L_rostralanteriorcingulate_cortex",
#                 66: "R_rostralanteriorcingulate_cortex",
#                 11: "L_lateralorbitofrontal_cortex", 
#                 52: "R_lateralorbitofrontal_cortex", 
#                 13: "L_medialorbitofrontal_cortex",
#                 54: "R_medialorbitofrontal_cortex",
#                 41: "L_amygdala",
#                 82: "R_amygdala",
#                 40: "L_hippocampus",
#                 81: "R_hippocampus",
#                 35: "L_thalamus",
#                 76: "R_thalamus"}

# removed orbitofrontal & caudal ACC
region_dict = { 
                6: "L_fusiform_cortex",
                23: "L_precentral_cortex",
                24: "L_precuneus_cortex",
                47: "R_fusiform_cortex",
                64: "R_precentral_cortex",
                65: "R_precuneus_cortex",
                25: "L_rostralanteriorcingulate_cortex",
                66: "R_rostralanteriorcingulate_cortex",
                41: "L_amygdala",
                82: "R_amygdala",
                40: "L_hippocampus",
                81: "R_hippocampus",
                35: "L_thalamus",
                76: "R_thalamus"}

print(region_dict)

    id                            hemi_label
25  25     L_rostralanteriorcingulate_cortex
35  35  L_thalamusproper_subcortex/brainstem
40  40     L_hippocampus_subcortex/brainstem
41  41        L_amygdala_subcortex/brainstem
66  66     R_rostralanteriorcingulate_cortex
76  76  R_thalamusproper_subcortex/brainstem
81  81     R_hippocampus_subcortex/brainstem
82  82        R_amygdala_subcortex/brainstem
{6: 'L_fusiform_cortex', 23: 'L_precentral_cortex', 24: 'L_precuneus_cortex', 47: 'R_fusiform_cortex', 64: 'R_precentral_cortex', 65: 'R_precuneus_cortex', 25: 'L_rostralanteriorcingulate_cortex', 66: 'R_rostralanteriorcingulate_cortex', 41: 'L_amygdala', 82: 'R_amygdala', 40: 'L_hippocampus', 81: 'R_hippocampus', 35: 'L_thalamus', 76: 'R_thalamus'}


In [10]:
# Define the data for columns 0 and 1: col 0 always has smaller id of the pair, col 1 always has larger id of the pair

## initially amy<>orbitofronal amy<>ACC
# col0 = [2, 25, 11, 13, 41, 41, 41, 41, 2, 25, 11, 13, 43, 66, 52, 54]  
# col1 = [41, 41, 41, 41, 43, 66, 52, 54, 82, 82, 82, 82, 82, 82, 82, 82] 

# added in hippocampus and thalamus
# col0 = [2, 25, 11, 13, 41, 41, 41, 41, 2, 25, 11, 13, 43, 66, 52, 54,
#        40, 81, 41, 41, 35, 35, 40, 76, 25, 25, 35, 66]  
# col1 = [41, 41, 41, 41, 43, 66, 52, 54, 82, 82, 82, 82, 82, 82, 82, 82,
#        41, 82, 81, 82, 40, 81, 76, 81, 35, 76, 66, 76] 

# removed orbitofrontal & caudal ACC
col0 = [25, 25, 41, 66, 40, 40, 41, 81, 
       25, 25, 35, 66, 35, 35, 40, 76]

col1 = [41, 82, 66, 82, 41, 82, 81, 82, 
       35, 76, 66, 76, 40, 81, 76, 81] 


# Create the DataFrame with columns 0 and 1
emo_roi_combo = pd.DataFrame({
    '0': col0,
    '1': col1
})
# combining ROI ids to make it easier to identify in next step
emo_roi_combo['combination'] = list(zip(emo_roi_combo.iloc[:, 0], emo_roi_combo.iloc[:, 1]))

print(emo_roi_combo)

     0   1 combination
0   25  41    (25, 41)
1   25  82    (25, 82)
2   41  66    (41, 66)
3   66  82    (66, 82)
4   40  41    (40, 41)
5   40  82    (40, 82)
6   41  81    (41, 81)
7   81  82    (81, 82)
8   25  35    (25, 35)
9   25  76    (25, 76)
10  35  66    (35, 66)
11  66  76    (66, 76)
12  35  40    (35, 40)
13  35  81    (35, 81)
14  40  76    (40, 76)
15  76  81    (76, 81)


## Loading n=967 subject IDs, creating a dictionary to hold DFs for each subject, filtering only emo-related ROI pairings

In [11]:
# load subject IDs 
subjects646_csv = pd.read_csv("subjects_967.csv")

# extract subject numbers to list (I suggest using 5 IDs to test  ex. [:5, 0])
subjects646 = subjects646_csv.iloc[:, 0].tolist() 
# print(len(subjects_ID))

In [12]:
# initialize dictionary to store DFs for each subject
filtered_dfs = {}

# loop through subjects, create DFs, store in above dictionary
for subject in subjects646:
    # create & read csv for each subject & rename 3rd col sub specific 
    csv_path =  hcp_base_path / "hcp1200" / f"sub-{subject}" / "ses-1" / "connectomes" / "DKT_space-MNI152NLin6_res-1x1x1" / f"sub-{subject}_ses-1_run-1_dwi_DKT_space-MNI152NLin6_res-1x1x1_connectome.csv"
    # Check if the file exists
    if not csv_path.exists():
        print(f"File not found for subject {subject}, skipping...")
        continue
    
    try:
        # Attempt to read the CSV file
        df = pd.read_csv(csv_path.fspath, sep=" ", header=None)
        df = df.rename(columns={df.columns[2]: f"{subject}_tracts"})
    except pd.errors.EmptyDataError:
        print(f"File for subject {subject} is empty, skipping...")
        continue
    
    # Create a combination of columns 0 and 1 in df for filtering (this is to compare to the combination made prior)
    df['combination'] = list(zip(df.iloc[:, 0], df.iloc[:, 1]))
    
    # Filter df to only keep rows where combination is in emo_roi_combo
    filtered_rois = df[df['combination'].isin(emo_roi_combo['combination'])]
    
    # Drop the 'combination' helper column after filtering
    filtered_rois = filtered_rois.drop(columns=['combination'])
    
    # Store the filtered DataFrame in the filtered_dfs dictionary
    filtered_dfs[subject] = filtered_rois

File for subject 106824 is empty, skipping...
File not found for subject 107220, skipping...
File not found for subject 109325, skipping...
File for subject 111413 is empty, skipping...
File not found for subject 116120, skipping...
File not found for subject 117728, skipping...
File for subject 118932 is empty, skipping...
File not found for subject 121315, skipping...
File not found for subject 121820, skipping...
File not found for subject 123925, skipping...
File for subject 125222 is empty, skipping...
File not found for subject 125525, skipping...
File not found for subject 126931, skipping...
File not found for subject 128127, skipping...
File not found for subject 128329, skipping...
File not found for subject 129129, skipping...
File not found for subject 129432, skipping...
File not found for subject 129533, skipping...
File not found for subject 130922, skipping...
File not found for subject 131621, skipping...
File not found for subject 137128, skipping...
File not found fo

In [13]:
# filtered_dfs 

## Extract tract column from each subject and save in a DF

In [33]:
# extract 1st DF from dictionary 
first_df = next(iter(filtered_dfs.values()))
# copy into final DF
final_df = first_df.iloc[:, :3].copy()
final_df

,0,1,100206_tracts
774,25,35,56.0
778,25,41,582.0
871,35,40,1134.0
880,35,81,1.0
957,40,41,230.0
1471,66,76,31.0
1475,66,82,871.0
1553,76,81,191.0
1567,81,82,131.0


In [34]:
# chatCPT extraordinaire 
final_df.columns = [0, 1, f"{next(iter(filtered_dfs.keys()))}"]

# Loop through the dictionary and merge each DataFrame's tracts column into final_df
for subject, df in filtered_dfs.items():
    if subject != next(iter(filtered_dfs.keys())):  # Skip the first subject already in final_df
        # Prepare the DataFrame to merge
        merge_df = df.iloc[:, :2].copy()  # Take the first two columns (col0 and col1)
        merge_df[f"{subject}"] = df.iloc[:, 2]  # Add the tracts column with subject-specific name
        
        # Merge with final_df based on match to columns 0 and 1
        final_df = pd.merge(final_df, merge_df, on=[0, 1], how='outer')

final_df.head()

,0,1,100206,100307,100408,100610,101006,101107,101309,101410,101915,102008,102109,102311,102513,102614,102715,102816,103010,103111,103212,103414,103515,103818,104012,104416,104820,105014,105115,105216,105923,106319,106521,107018,107321,107422,107725,108020,108121,108222,108525,108828,109123,110007,110411,111009,111211,111514,112314,112516,112819,112920,113215,113316,113619,113922,114116,114217,114318,114419,114621,114823,115017,115219,115320,115724,115825,116221,116423,116524,117021,117122,117324,117930,118023,118124,118225,118730,118831,119025,119126,119732,119833,120111,120212,120414,120717,121416,121618,121921,122317,122822,123117,123420,123521,123723,123824,124220,124624,124826,125424,126325,126426,126628,127226,127630,127731,127832,127933,128026,128632,128935,129028,129331,129634,129937,130013,130114,130316,130417,130619,130720,130821,131217,131419,131722,131823,131924,132017,132118,133019,133625,133827,133928,134021,134223,134324,134425,135124,135225,135528,135629,135730,135932,136227,136732,136833,137027,137229,137431,137532,137633,137936,138231,138332,138534,138837,139435,139839,140319,140420,140824,140925,141119,141422,143224,143325,143426,144125,144226,144428,144731,144933,145834,146331,146432,146533,146634,146735,146937,147030,147636,148032,148133,148335,148941,149236,149337,149539,149842,150524,150726,150928,151021,151223,151324,151425,151526,151627,151829,151930,152225,152427,152831,153025,153126,153227,153429,153631,153732,153833,153934,154229,154330,154431,154532,154734,154835,154936,155231,155938,156031,156233,156334,156536,156637,157336,157437,158035,158136,158338,158540,159138,159239,159340,159441,159744,159946,160123,160729,160830,160931,161327,161731,161832,162026,162228,162329,162733,162935,163129,163331,163836,164030,164131,164636,164939,165032,165436,165638,165840,166640,167036,167238,167440,167743,168240,168341,168745,168947,169040,169343,169444,169545,169747,169949,170934,171330,171431,171532,171633,172029,172130,172332,172433,172534,172635,172938,173132,173233,173334,173435,173536,173637,173738,173839,173940,174437,174841,175136,175237,175338,175439,175540,175742,176037,176441,176542,176845,177342,177645,177746,178142,178748,178950,179245,179346,179548,179952,180129,180230,180432,180533,180735,180937,181636,182032,182436,182739,182840,183034,183337,183741,185139,185341,185442,185846,185947,186141,186444,186545,186848,187143,187345,187547,187850,188145,188347,188448,188549,189349,189450,190031,191033,191235,191336,191437,191942,192035,192136,192237,192540,192641,192843,193239,193441,193845,194140,194443,194645,194746,195445,195647,195950,196144,196346,196750,196851,196952,197348,197550,197651,198047,198350,198451,198653,198855,199150,199251,199352,199453,200008,200109,200311,200513,200614,200917,201111,201414,201818,202113,202719,203923,204016,204319,204420,204521,204622,205220,205725,206929,207123,207426,208024,208125,208226,208327,208428,208630,209127,209228,209329,209834,209935,210011,210112,210415,210617,211114,211316,211417,211619,211821,211922,212015,212116,212217,212318,212419,212823,213017,213421,213522,214019,214221,214423,214524,214625,214726,217126,217429,219231,220721,221319,223929,228434,233326,236130,237334,239136,239944,246133,248238,248339,249947,250427,250932,251833,255639,255740,256540,257542,257845,263436,268749,268850,274542,275645,280739,280941,281135,283543,284646,285345,285446,286347,286650,287248,289555,290136,293748,295146,298051,298455,299154,300618,300719,303119,303624,304020,304727,305830,307127,308129,309636,310621,311320,314225,316633,316835,317332,318637,320826,321323,322224,325129,329440,329844,330324,333330,334635,341834,342129,346137,346945,348545,349244,350330,351938,352132,352738,353740,355239,356948,358144,361234,361941,362034,365343,366042,366446,368551,368753,371843,376247,377451,378756,378857,379657,380036,381038,381543,385450,387959,389357,390645,391748,392447,393247,394956,395251,395756,395958,397154,397760,397861,406432,406836,412528,414229,419239,421226,42263

In [35]:
# add in the labels for the regions
final_df[0] = final_df[0].map(region_dict)
final_df[1] = final_df[1].map(region_dict)

final_df.head()

,0,1,100206,100307,100408,100610,101006,101107,101309,101410,101915,102008,102109,102311,102513,102614,102715,102816,103010,103111,103212,103414,103515,103818,104012,104416,104820,105014,105115,105216,105923,106319,106521,107018,107321,107422,107725,108020,108121,108222,108525,108828,109123,110007,110411,111009,111211,111514,112314,112516,112819,112920,113215,113316,113619,113922,114116,114217,114318,114419,114621,114823,115017,115219,115320,115724,115825,116221,116423,116524,117021,117122,117324,117930,118023,118124,118225,118730,118831,119025,119126,119732,119833,120111,120212,120414,120717,121416,121618,121921,122317,122822,123117,123420,123521,123723,123824,124220,124624,124826,125424,126325,126426,126628,127226,127630,127731,127832,127933,128026,128632,128935,129028,129331,129634,129937,130013,130114,130316,130417,130619,130720,130821,131217,131419,131722,131823,131924,132017,132118,133019,133625,133827,133928,134021,134223,134324,134425,135124,135225,135528,135629,135730,135932,136227,136732,136833,137027,137229,137431,137532,137633,137936,138231,138332,138534,138837,139435,139839,140319,140420,140824,140925,141119,141422,143224,143325,143426,144125,144226,144428,144731,144933,145834,146331,146432,146533,146634,146735,146937,147030,147636,148032,148133,148335,148941,149236,149337,149539,149842,150524,150726,150928,151021,151223,151324,151425,151526,151627,151829,151930,152225,152427,152831,153025,153126,153227,153429,153631,153732,153833,153934,154229,154330,154431,154532,154734,154835,154936,155231,155938,156031,156233,156334,156536,156637,157336,157437,158035,158136,158338,158540,159138,159239,159340,159441,159744,159946,160123,160729,160830,160931,161327,161731,161832,162026,162228,162329,162733,162935,163129,163331,163836,164030,164131,164636,164939,165032,165436,165638,165840,166640,167036,167238,167440,167743,168240,168341,168745,168947,169040,169343,169444,169545,169747,169949,170934,171330,171431,171532,171633,172029,172130,172332,172433,172534,172635,172938,173132,173233,173334,173435,173536,173637,173738,173839,173940,174437,174841,175136,175237,175338,175439,175540,175742,176037,176441,176542,176845,177342,177645,177746,178142,178748,178950,179245,179346,179548,179952,180129,180230,180432,180533,180735,180937,181636,182032,182436,182739,182840,183034,183337,183741,185139,185341,185442,185846,185947,186141,186444,186545,186848,187143,187345,187547,187850,188145,188347,188448,188549,189349,189450,190031,191033,191235,191336,191437,191942,192035,192136,192237,192540,192641,192843,193239,193441,193845,194140,194443,194645,194746,195445,195647,195950,196144,196346,196750,196851,196952,197348,197550,197651,198047,198350,198451,198653,198855,199150,199251,199352,199453,200008,200109,200311,200513,200614,200917,201111,201414,201818,202113,202719,203923,204016,204319,204420,204521,204622,205220,205725,206929,207123,207426,208024,208125,208226,208327,208428,208630,209127,209228,209329,209834,209935,210011,210112,210415,210617,211114,211316,211417,211619,211821,211922,212015,212116,212217,212318,212419,212823,213017,213421,213522,214019,214221,214423,214524,214625,214726,217126,217429,219231,220721,221319,223929,228434,233326,236130,237334,239136,239944,246133,248238,248339,249947,250427,250932,251833,255639,255740,256540,257542,257845,263436,268749,268850,274542,275645,280739,280941,281135,283543,284646,285345,285446,286347,286650,287248,289555,290136,293748,295146,298051,298455,299154,300618,300719,303119,303624,304020,304727,305830,307127,308129,309636,310621,311320,314225,316633,316835,317332,318637,320826,321323,322224,325129,329440,329844,330324,333330,334635,341834,342129,346137,346945,348545,349244,350330,351938,352132,352738,353740,355239,356948,358144,361234,361941,362034,365343,366042,366446,368551,368753,371843,376247,377451,378756,378857,379657,380036,381038,381543,385450,387959,389357,390645,391748,392447,393247,394956,395251,395756,395958,397154,397760,397861,406432,406836,412528,414229,419239,421226,42263

In [36]:
# print(final_df)

In [37]:
# save DF as CSV & transpose
output_csv_path = "DTI_DF.csv"
final_df.T.to_csv(output_csv_path)

In [38]:
final_df.shape

(16, 888)